In [147]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import pandas as pd
import pyautogui
import getpass
import time
import urllib.parse
import traceback
import os

In [148]:
'''
TO ADD MORE PAPERS ADD THE FILEPATH TO THE .CSV FILE AND GIVE IT A VARIABLE NAME
'''
DATAFILE_PATH = "third_dataset.csv"
STARTER_LINK ='https://my.usc.edu/service/library/'
REFERNCE_FILE = 'extracted_unique_doi_links.csv'
TITLE_REFERENCE_FILE = 'references_titles_RG.csv'

In [149]:
def encodeString(string):
    newStr = string.encode(encoding = 'UTF-8', errors = 'strict')
    return newStr

def decodeString(byte_string):
    decoded_str = byte_string.decode(encoding='UTF-8', errors='strict')
    return decoded_str

def makefileName(title):
    return title.replace("/"," ").replace(" ","_").replace(".", '_').replace(".", '_')

In [150]:
class Paper:
    def __init__(self, doi, codedDOI="None", title="None", authors="None"):
        self.doi = doi
        self.encodedDOI = codedDOI
        self.title = title
        self.authors = authors
        self.table = None

    def getCoded(self):
        return self.encodedDOI

    def getContent(self):
        return self.table

    def getAuthors(self):
        return self.authors

    def getTitle(self):
        return self.title

    def getDOI(self):
        return self.doi
    
    def setContent(self, table):
        self.table = table

In [151]:
'''
2. READ YOUR CSV INTO A DICTIONARY THAT IS STRUCTURED LIKE THIS:
    paper_dictionary = {"doi1" : Paper Class Object, "doi2" : Paper Class Object, ..}

    IF NEEDED:
    COPY THESE FUNCTIONS TO CREATE PAPER objects to hold information about research papers that you want to download.
Keep the key for each one the 'doi' of that paper.

'''

# Get research papers of bik
def createPaperObjects(filedata_path):
    papers = {}
    df = pd.read_csv(filedata_path)
    rows = df.shape[0]
    for i in range(rows):
        test_doi = df.loc[i, ["DOI"]][0]
        title = df.loc[i, ["Title"]][0]
        authors = df.loc[i, ['Authors']][0]
        papers[test_doi] = Paper(test_doi, encodeString(test_doi),title, authors)
    return papers

# get reference papers
def createReferencePapers(filedataPath):
    links = pd.read_csv(filedataPath)['DOI Links']
    references = {}
    for reference in links:
        # CONVERT THE HHTTP LINKS INTO STRING
        references[urllib.parse.unquote(reference)] = Paper(doi=urllib.parse.unquote(reference))
    return references

def createReferencesFromTitle(filedataPath):
    title_names = pd.read_csv(filedataPath)['Reference Title']
    titles = {}
    for title in title_names:
        titles[encodeString(title)] = Paper(doi=encodeString(title))
    return titles

papers = createPaperObjects(DATAFILE_PATH)
referencePapers = createReferencePapers(REFERNCE_FILE)
referencedTitlePaper = createReferencesFromTitle(TITLE_REFERENCE_FILE)

In [152]:
def getButton(title, driver, xpath):
    try:
        button=driver.find_element("xpath", xpath)
        return button
    except Exception as error:
        print(f"Failed to find button {title}")
        raise Exception(error)

In [153]:
def goToLogin(driver):
    signIn = getButton("SignIn Button",driver,'//a[@class="main-navigation__navbar "]/span[text()="Sign In"]')
    signIn.click()
    return driver

In [154]:
def douLogin(driver):
    username_slot = getButton("Username Button", driver, "//input[@id='username']")
    uid= input('Enter the User id: ')
    username_slot.send_keys(uid)
    
    password_slot = getButton("Password Button", driver, "//input[@id='password']")
    pswd = getpass.getpass('Password:')
    password_slot.send_keys(pswd)

    try:
        submit_button = getButton("Submit Button", driver, "//button[@type='submit']")
        submit_button.click()
    except Exception as e:
        print('Wrong user id or password')
        raise Exception(e)

    time.sleep(2)
    iframe='duo_iframe'
    driver.switch_to.frame(iframe)
    time.sleep(10)
    return driver

In [155]:
def downloadPaper(driver, counter, prefix):
    failed = False

    # Tru to see if there is a download button
    try:
        download_button=driver.find_element("xpath", "//button[@id='documentDownload']")
        download_button.click()
    except Exception:
        failed = True

    if failed:
        # try if it can be stored as PDF
        try:
            pyautogui.hotkey('ctrl', 's')
            time.sleep(1)
            pyautogui.write(f'{prefix}_{counter}')
            pyautogui.press('enter')
            failed = False
        except Exception:
            failed = True

    if failed:
        print("Failed All attempts to download")


In [156]:
def searchByDOI(driver, doi):
    search_bar = getButton("For Search Bar",driver,'//input[@id="library-query"]')
    search_bar.clear()
    search_bar.send_keys(doi)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(2)
    return driver 

In [157]:
def closeTabs(driver):
    all_tabs = driver.window_handles
    for tab in all_tabs[1:]:
        driver.switch_to.window(tab)
        driver.close()
    driver.switch_to.window(driver.window_handles[0])
    return driver

def findRightPaper(driver, doi, counter, prefix):
    try:
        driver = searchByDOI(driver, doi)
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(6)
        xpath_download_button = "//a[contains(@class, 'article-link')]//span[@class='dl-article']"
        download_button = getButton("Library Download Button", driver,xpath_download_button)
        download_button.click()
        # Switch to new tab
        driver.switch_to.window(driver.window_handles[2])
        time.sleep(15) # give time to load
        downloadPaper(driver, counter, prefix)
        time.sleep(2)
        driver = closeTabs(driver)
        return driver, True
    except Exception as error:
        driver = closeTabs(driver)
        return driver, False

In [158]:
def preStartUp():
    driver = uc.Chrome()
    time.sleep(2)
    driver.get(STARTER_LINK)
    driver.maximize_window()
    driver = douLogin(driver)
    time.sleep(10)
    return driver

In [159]:
def GetAllPaper(tracker, tracker_file, listItems, prefix, encoded=False):
    if os.path.isfile(tracker_file):
        with open(tracker_file, 'r') as f:
            tracker = eval(f.read())
    try:
        driver = preStartUp()
        if driver.current_url != STARTER_LINK:
            driver.get(STARTER_LINK)
        
        # Switch to the right tab (REQUIRED HERE)
        switch_to_search_tab = driver.find_element(By.XPATH, "//li[@class='search']//a")
        switch_to_search_tab.click()
        time.sleep(2)
        counter = 0
        for doi, obj in listItems.items():
            counter += 1
            try:
                if (tracker[doi] == 0):
                    print(f"Searching for: {doi}")
                    if encoded:
                        driver, downloaded = findRightPaper(driver, decodeString(obj.getDOI()), counter, prefix)
                    else:
                        driver, downloaded = findRightPaper(driver, obj.getDOI(), counter, prefix)
                    if downloaded:
                        tracker[doi] = 1
                else:
                    print(f"Done: {doi}")
            except Exception as e:
                driver = closeTabs(driver)
                print(f"Failed: {doi}")
    except Exception as err:
        with open(tracker_file, 'w') as f:
            f.write(str(tracker))
        raise Exception(err)
    finally:
        with open(tracker_file, 'w') as f:
            f.write(str(tracker))

In [160]:
# tracked_done = {}
# for doi, obj in papers.items():
#     tracked_done[doi] = 0
# try:
#     GetAllPaper(tracked_done, "downloaded_papers.txt", papers, "ResearchPaper_")
# except Exception:
#     traceback.print_exc()

In [161]:
tracked_done_title_reference = {}
for doi, obj in referencedTitlePaper.items():
    tracked_done_title_reference[doi] = 0
try:
    GetAllPaper(tracked_done_title_reference, "downloaded_papers_title_reference.txt", referencedTitlePaper, "ResearchPaperTitleRef_", encoded=True)
except Exception:
    traceback.print_exc()

Searching for: b'Microinjection of colchicine into sea urchin eggs'
Searching for: b'Control of Cleavage Spindle Orientation in Caenorhabditis Elegans: The Role of the Genes Par-2 and Par-3'
Searching for: b'Mating and tetrad analysis in Chlamydomonas reinhardtii'
Searching for: b'Transient localized accumulation of actin in Caenorhabditis elegans blastomeres with oriented asymmetric divisions'
Searching for: b'The JNM1 gene in the yeast Saccharomyces cerevisiae is required for nuclear migration and spindle orientation during the mitotic cell cycle'
Searching for: b'Disruption of mitotic spindle orientation in a yeast dynein mutant'
Searching for: b'A mutation in the \xce\xb11-tubulin gene of Chlamydomonas reinhardtii confers resistance to anti-microtubule herbicides'
Searching for: b'A mating type-linked gene cluster expressed in Chlamydomonas zygotes participates in the uniparental inheritance of the chloroplast genome'
Searching for: b'Determination of cleavage planes'
Searching for